In [ ]:
import os
import sys
import glob
import numpy as np
from scipy.signal import welch
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, NullLocator, FixedFormatter
fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Arial', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})
OU_tau = 20e-3

In [ ]:
network = 'SM_with_load'
if network == 'IEEE39':
    PF_net_name = 'IEEE39_stoch'
    condition = 'default'
    load_type = 'static_const_Z'
    load_name = 'Load_03'
    expt_name = os.path.join(condition,load_type,load_name)
    fmin, fmax, steps_per_decade = -6, 2, 1000
    outfile = 'TF_{}-{}-{}-{}'.format(PF_net_name, condition, load_type, load_name)
elif network == 'Sardinia':
    static_load = True
    if static_load:
        negative_load = False
        if negative_load:
            PF_net_name = 'V2020_Rete_Sardegna_2021_06_03cr_stoch'
        else:
            PF_net_name = 'V2020_Rete_Sard_2021_06_03cr_mod_loads'
        condition = 'default'
        load_name = 'EqX_NARCDI1501TRB_____LOAD____'
        dP = 0.1
        if negative_load:
            subdir = os.path.join('negative_load',f'dP_{dP:g}')
        else:
            subdir = os.path.join('positive_load',f'dP_{dP:g}_with_fe')
    else:
        PF_net_name = 'V2020_Rete_Sardegna_2021_06_03cr_stoch'
        condition = 'default'
        load_name = 'EqX_BNFC_I0601TRR_____LOAD____'
        dP = 0.01
        subdir = os.path.join(f'dP_{dP}')
    expt_name = os.path.join(condition, load_name, subdir)
    fmin, fmax, steps_per_decade = -6, 2, 100
    outfile = 'TF_{}-{}-{}'.format(PF_net_name, load_name, subdir.split(os.path.sep)[-1])
elif network in ('SM_with_load','SM_with_line_and_load','SM_with_line_and_loads'):
    PF_net_name = network
    load_type = 'static_load_const_Z_2'
    expt_name = load_type
    if 'loads' in network:
        expt_name = os.path.join(expt_name, 'LD1')
    fmin, fmax, steps_per_decade = -6, 2, 1000
    outfile = 'TF_{}-{}'.format(PF_net_name, expt_name.replace(os.path.sep,'-'))
else:
    raise Exception(f'Unknown network: "{network}"')
folder = os.path.join('..','..','modal_analysis',network,expt_name)
folder = os.path.join('..','data',network,expt_name)
if not os.path.isdir(folder):
    raise Exception(f'{folder}: no such folder')
outfile = os.path.join(folder, outfile)
AC_data_file = os.path.join(folder, f'{PF_net_name}_AC_TF_{fmin:.1f}_{fmax:.1f}_{steps_per_decade}.npz')
AC_data = np.load(AC_data_file, allow_pickle=True)
dB = AC_data['dB'].item()

In [ ]:
AC_data.files

In [ ]:
TF2 = AC_data['TF2']
mag2 = dB * np.log10(np.abs(TF2))

In [ ]:
magur = dB * np.log10(np.abs(TF2)[:,0])
plt.semilogx(AC_freq, magur)

In [ ]:
discard = 300 # [s]
tran_data_files = sorted(glob.glob(os.path.join(folder,f'{PF_net_name}_tran_*.npz')))
tran_data_files = tran_data_files[:2]
print(f'Found {len(tran_data_files)} data files.')
tran_blobs = [np.load(f, allow_pickle=True) for f in tran_data_files]
time = [blob['time'] for blob in tran_blobs]
var_name = 'm:ur:bus1' #'s:xspeed'
if var_name == 'V':
    X = [np.abs(blob['data'].item()['gen']['m:ur:bus1'] + \
                1j*blob['data'].item()['gen']['m:ui:bus1']) for blob in tran_blobs]
else:
    X = [blob['data'].item()['gen'][var_name] for blob in tran_blobs]
if X[0].ndim == 1:
    X = [np.reshape(x, (-1,1)) for x in X]
X = np.concatenate([x[t>discard,:] for t,x in zip(time, X)], axis=0)
dt = time[0][1] - time[0][0]
tran_time = np.arange(X.shape[0]) * dt

In [ ]:
try:
    fe = [blob['data'].item()['bus']['m:fe'] for blob in tran_blobs]
    tran_fe = np.concatenate([f[t>discard,:] for t,f in zip(time,fe)], axis=0)
    print('Frequency saved in data files.')
except:
    pass

In [ ]:
AC_M = AC_data['Mtot'].item()
M = tran_blobs[0]['momentum'].item()
E = tran_blobs[0]['energy'].item()

In [ ]:
# fref = 1
# Δω = (tran_speed - 1) * fref
ΔX = X-1
window_dur = 60 * 15
window = window_dur / dt
onesided = True
tran_freq,tran_Pxx = welch(ΔX, 1/dt, window='boxcar', nperseg=window, noverlap=window/2,
                           nfft=window, return_onesided=True, scaling='density', axis=0)
tran_Pxx /= 2
tran_freq,tran_Pxx = tran_freq[1:],tran_Pxx[1:,:]

In [ ]:
try:
    _,tran_fe_Pxx = welch(tran_fe-1, 1/dt, window='boxcar', nperseg=window, noverlap=window/2,
                          nfft=window, return_onesided=True, scaling='density', axis=0)
    tran_fe_Pxx /= 2
    tran_fe_Pxx = tran_fe_Pxx[1:,:]
except:
    pass

In [ ]:
from scipy.io import savemat
out = {}
for key in AC_data.files:
    if key in ('F','TF','mag','phase'):
        out['AC_'+key] = AC_data[key]
    else:
        out[key] = AC_data[key]
out['tran_F'] = tran_freq
out['tran_mag'] = 10 * np.log10(tran_Pxx)
np.savez_compressed(outfile + '.npz', **out)
savemat(outfile + '.mat', out)

In [ ]:
device_names = tran_blobs[0]['device_names'].item()['gen']
if len(device_names) == 10:
    rows,cols = 2,5
    w,h = 3,2.5
elif len(device_names) == 18:
    rows,cols = 3,6
    w,h = 3,2.5
elif len(device_names) == 1:
    rows,cols = 1,1
    w,h = 5,4

# pretty-plot
pp = True
if pp:
    if len(device_names) == 1:
        w,h = 10/2.54, 6/2.54
    else:
        rows,cols = 2,2
        w,h = 7.5/2.54, 5/2.54

fig,ax = plt.subplots(rows, cols, figsize=(cols*w, rows*h), sharex=True, sharey=True, squeeze=False)
ticks = np.logspace(-3, 2, 6)
F0 = 0.1
red,green,magenta,orange = [0.75,0,0], [0,.75,0], [.75,0,.75], [1,.5,0]
col = [green,magenta]
cmap = plt.get_cmap('tab10', len(AC_data))
lw = 1
AC_freq = AC_data['F']
if pp and len(device_names) == 18:
    devices_to_plot = ['FSACTI0201GGR3____GEN_____', 'BNFC_I0601GGR1____GEN_____',
                       'SULCTI0202GGR2____GEN_____', 'FL1CZI0101GGR1____GEN_____']
else:
    devices_to_plot = device_names[:rows*cols]
for k,name in enumerate(devices_to_plot):
    i,j = k//cols, k%cols
    try:
        ### 1
        idx = device_names.index(name)
        y = 10 * np.log10(tran_Pxx[:,idx])
        ym,yM = y.min(),y.max()
        ax[i,j].semilogx(tran_freq, y, 'k', lw=lw, label=r'TRAN: E = {:.2f} GW$\cdot$s'.format(E*1e-3))
        ### 2
#         idx = np.where(AC_data['SM_names'] == name)[0][0]
#         y = AC_data['mag'][:,idx]
#         if y.min() < ym:
#             ym = y.min()
#         if y.max() > yM:
#             yM = y.max()
#         ax[i,j].semilogx(AC_freq, y, color=red, lw=1.5, label='AC')
        ### 3
        ax[i,j].plot(1/(2*np.pi*OU_tau)+np.zeros(2), [ym,yM], ':', color=orange, lw=2, label='OU cutoff')
        ax[i,j].set_title(name.split('___')[0], fontsize=8)
        ax[i,j].xaxis.set_major_locator(FixedLocator(ticks))
        ax[i,j].xaxis.set_minor_locator(NullLocator())
        ax[i,j].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
        ax[i,j].set_xlim(ticks[[0,-1]])
    except:
        print(f'Device name {name} missing')
ax[0,0].legend(loc='lower left', frameon=False, fontsize=6)
for a in ax[-1,:]:
    a.set_xlabel('Frequency [Hz]')
for a in ax[:,0]:
    a.set_ylabel(f'PSD [dB{dB}]')
sns.despine()
fig.tight_layout()
plt.savefig(outfile + '.pdf')

In [ ]:
try:
    device_names = ['BOLC_I2201A1______BUS_____', 'BSACTI2201A1______BUS_____', 'CODCTI2201A1______BUS_____',
                'ECOCZI2201A1______BUS_____', 'EURC_I2201A1______BUS_____', 'FSACTI2201A1______BUS_____', 
                'GRIC_I2201A1______BUS_____', 'MIMC_I2201A1______BUS_____', 'MOGC_I2201A1______BUS_____', 
                'ORSCTI2201A1______BUS_____', 'OTTCTI2201A1______BUS_____', 'PSOCTI2201A1______BUS_____', 
                'RUMCTI2201A1______BUS_____', 'SELCTI2201A1______BUS_____', 'SULCTI2201A1______BUS_____',
                'TALCTI2201A1______BUS_____', 'TIRC_I2201A1______BUS_____', 'VORCTI2201A1______BUS_____']
    rows,cols = 2,2
    w,h = 7.5/2.54, 5/2.54

    fig,ax = plt.subplots(rows, cols, figsize=(cols*w, rows*h), sharex=True, sharey=True, squeeze=False)
    ticks = np.logspace(-3, 2, 6)
    cmap = plt.get_cmap('tab10', len(device_names))
    devices_to_plot = [device_names[i] for i in [5,14,2,16]]
    for k,name in enumerate(devices_to_plot):
        i,j = k//cols, k%cols
        try:
            ### 1
            idx = device_names.index(name)
            y = 10 * np.log10(tran_fe_Pxx[:,idx])
            ym,yM = y.min(),y.max()
            ax[i,j].semilogx(tran_freq, y, 'k', lw=lw, label=r'TRAN: E = {:.2f} GW$\cdot$s'.format(E*1e-3))
            ### 3
            ax[i,j].plot(1/(2*np.pi*OU_tau)+np.zeros(2), [ym,yM], ':', color=orange, lw=2, label='OU cutoff')
            ax[i,j].set_title(name.split('___')[0], fontsize=8)
            ax[i,j].xaxis.set_major_locator(FixedLocator(ticks))
            ax[i,j].xaxis.set_minor_locator(NullLocator())
            ax[i,j].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
            ax[i,j].set_xlim(ticks[[0,-1]])
        except:
            print(f'Device name {name} missing')
    ax[0,0].legend(loc='lower left', frameon=False, fontsize=6)
    for a in ax[-1,:]:
        a.set_xlabel('Frequency [Hz]')
    for a in ax[:,0]:
        a.set_ylabel(f'PSD [dB{dB}]')
    sns.despine()
    fig.tight_layout()
    plt.savefig(outfile + '_fe.pdf')
except:
    pass

In [ ]:
if False:
    N_gen = len(device_names)
    # groups = sorted(list(set([name[:2] for name in device_names])))
    groups = ['AS','BMA','BMS','BN','CO','FL','FS','OZ','SE','SL','SU','TI']
    N_groups = len(groups)
    cmap = sns.color_palette('tab20', n_colors=N_groups)
    fig,ax = plt.subplots(1, 1, figsize=(10/2.54,6/2.54))
    x = [0.0012, 0.03]
    y = np.linspace(-140, -195, 9)
    for k in range(N_gen):
        name = device_names[k].split('__')[0]
        if name[:2] == 'BM':
            col = cmap[groups.index(name[:3])]
        else:
            col = cmap[groups.index(name[:2])]
        ax.semilogx(AC_freq, AC_data['mag'][:,k], color=col, lw=0.75, label=name)
        i,j = k//9,k%9
        ax.text(x[i], y[j], name, fontsize=6, color=col)
    # ax.legend(loc='lower left', frameon=False, fontsize=6)
    ax.xaxis.set_major_locator(FixedLocator(ticks))
    ax.xaxis.set_minor_locator(NullLocator())
    ax.xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
    ax.set_xlim(ticks[[0,-1]])
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel('PSD [dB20]')
    sns.despine()
    fig.tight_layout()
    plt.savefig(outfile + '_all_gen.pdf')